# GAN
使用 PyTorch 实现一个简单的 GAN 网络，并测试该网络的效果。

首先导入相关库

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
from torch.autograd import Variable

定义 GAN 网络的两个组成部分：生成器和判别器。

生成器的定义如下：

In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim * 2)
        self.fc3 = nn.Linear(hidden_dim * 2, hidden_dim * 4)
        self.fc4 = nn.Linear(hidden_dim * 4, output_dim)
        self.activation = nn.ReLU()

    def forward(self, z):
        x = self.activation(self.fc1(z))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

生成器接收一个噪声向量 z，并输出一张图像。它包含了四个全连接层，每个层都使用了 ReLU 激活函数。

判别器的定义如下：

In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim * 4)
        self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim * 2)
        self.fc3 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)
        self.activation = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

判别器接收一张图像，并输出一个标量值，用于表示这张图像是否为真实图像。它包含了四个全连接层，前三个层都使用了 ReLU 激活函数，最后一层使用了 Sigmoid 激活函数。

In [ ]:
def train(generator, discriminator, dataloader, n_epochs, z_dim, lr, device):
    # 设置优化器
    optimizer_G = optim.Adam(generator.parameters(), lr=lr)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

    # 定义损失函数
    criterion = nn.BCELoss()

    # 开始训练
    for epoch in range(n_epochs):
        for i, (real_images, _) in enumerate(dataloader):
            real_images = real_images
            batch_size = real_images.shape[0]

            # 训练判别器
            optimizer_D.zero_grad()
            noise = torch.randn(batch_size, z_dim)
            fake_images = generator(noise)
            real_labels = torch.ones(batch_size, 1)
            